In [ ]:
!pip install scikit-learn==0.22.2.post1

In [ ]:
!pip install opencv-python

In [ ]:
import os 
import cv2

In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten, MaxPooling2D, Dropout
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import joblib
from keras.models import model_from_json

In [ ]:
import tensorflow
import keras
print("tensorflow",tensorflow.__version__)
print("keras",keras.__version__)

## Loading trained model

In [ ]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
labels_dict = {0 : 'anger', 1 : 'contempt', 2 : 'disgust', 3 : 'fear', 4 : 'happy',5 :'sadness',6 :'surprise'}
Gender_dict = {0:'female',1:'male'}

In [ ]:
scaler = joblib.load('scaler.pkl')

In [ ]:
json_file = open('emotion_model.json', 'r')
emotion_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(emotion_model_json)
# load weights into new model
emotion_model.load_weights("emotion_model.h5")
print("Loaded model from disk")

In [ ]:
json_file = open('age&gender_model.json', 'r')
Age_Gender_json = json_file.read()
json_file.close()
Age_Gender_model = model_from_json(Age_Gender_json)
# load weights into new model
Age_Gender_model.load_weights("age&gender_model.h5")
print("Loaded model from disk")

In [ ]:
def detect(im):
    faces = face_clsfr.detectMultiScale(im)

    for (x, y, w, h) in faces:
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(224,224))
        resized = resized/255
        reshaped=np.expand_dims(resized, axis = 0)
        result=emotion_model.predict(reshaped)
        label=np.argmax(result)


        img_age_gender = cv2.resize(face_img[:,:,0],dsize=(64,64))
        img_age_gender = img_age_gender.reshape((img_age_gender.shape[0],img_age_gender.shape[1],1))
        img_age_gender = np.expand_dims(img_age_gender, axis = 0)
        img_age_gender = img_age_gender/255

        age = str(int(np.round(scaler.inverse_transform(Age_Gender_model.predict(img_age_gender)[0])[0][0])))

        gender_prob = Age_Gender_model.predict(img_age_gender)[1][0][0]
        if gender_prob < 0.5:
            gender = 0
        else:
            gender = 1

        gender = Gender_dict[gender]

        cv2.rectangle(im,(x,y),(x+w,y+h),(0, 0, 225),2)
        cv2.putText(im,labels_dict[label] , (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        cv2.putText(im,"Gender: "+gender , (x, y-30),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        cv2.putText(im,"Age: " + age , (x, y-50),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    return im

# Detection from realtime

In [ ]:

video_capture = cv2.VideoCapture(0) 

while True: 
    _, frame = video_capture.read()
    frame=cv2.flip(frame,1,1)
    frame = detect(frame)
    cv2.imshow('Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 

video_capture.release()
cv2.destroyAllWindows() 